In [1]:
import torch
import scipy.io as sio
import numpy as np
import pickle

### KAGGLE DATASET - IMPORT AND COMPILE IMAGES WITH THEIR SENTENCE DESCRIPTIONS

In [2]:
# Check that MPS is available
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    mps_device = torch.device("mps")

In [3]:
class kaggleImage:
    def __init__(self):
        self.image_id = None
        self.image_class_id = None
        self.image_class_name = None
        self.image_sentences = None
        self.is_train = None
        self.image_index = None
        self.image_tensor_64 = None
        self.image_tensor_128 = None
        self.image_tensor_256 = None
        
    def __str__(self):
        print_string = (f"Kaggle Image:\n"
              f"\tImage Id: {self.image_id}\n"
              f"\tImage Class ID: {self.image_class_id}\n"
              f"\tImage Class Name: {self.image_class_name}\n"
              f"\tImage Sentences: {self.image_sentences}\n"
              f"\tIs Train: {self.is_train}\n"
              f"\tImage shapes:\n"
              f"\t\t{self.image_tensor_64.shape}\n"
              f"\t\t{self.image_tensor_128.shape}\n"
              f"\t\t{self.image_tensor_256.shape}")
        
        return print_string

In [4]:
metadata = torch.load("/Users/lkk/Documents/BOUN CMPE/CMPE 537-Computer Vision/Term Project/Code/Datasets/CUB_Kaggle/metadata.pth")

/var/folders/w_/qpdjg8096dl3n3gz3wz38ksc0000gn/T/ipykernel_7264/2933442925.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  metadata = torch.load("/Users/lkk/Documents/BO

In [5]:
train_images_256 = torch.load("/Users/lkk/Documents/BOUN CMPE/CMPE 537-Computer Vision/Term Project/Code/Datasets/CUB_Kaggle/imgs_train_val_256x256.pth")
train_images_128 = torch.load("/Users/lkk/Documents/BOUN CMPE/CMPE 537-Computer Vision/Term Project/Code/Datasets/CUB_Kaggle/imgs_train_val_128x128.pth")
train_images_64 = torch.load("/Users/lkk/Documents/BOUN CMPE/CMPE 537-Computer Vision/Term Project/Code/Datasets/CUB_Kaggle/imgs_train_val_64x64.pth")

test_images_256 = torch.load("/Users/lkk/Documents/BOUN CMPE/CMPE 537-Computer Vision/Term Project/Code/Datasets/CUB_Kaggle/imgs_test_256x256.pth")
test_images_128 = torch.load("/Users/lkk/Documents/BOUN CMPE/CMPE 537-Computer Vision/Term Project/Code/Datasets/CUB_Kaggle/imgs_test_128x128.pth")
test_images_64 = torch.load("/Users/lkk/Documents/BOUN CMPE/CMPE 537-Computer Vision/Term Project/Code/Datasets/CUB_Kaggle/imgs_test_64x64.pth")

/var/folders/w_/qpdjg8096dl3n3gz3wz38ksc0000gn/T/ipykernel_7264/1046569069.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_images_256 = torch.load("/Users/lkk/Docu

In [6]:
images_kaggle = {}

for i in range(len(metadata["img_ids"])):
    image = kaggleImage()
    image.image_id = metadata["img_ids"][i]
    image.image_class_id = metadata["img_id_to_class_id"][image.image_id]
    image.image_class_name = metadata["class_id_to_class_name"][image.image_class_id]
    
    image_sentences = []
    for encoded_sentence in metadata["img_id_to_encoded_caps"][image.image_id]:
        decoded_sentence = ""
        for encoded_word in encoded_sentence:
            decoded_word = metadata["word_id_to_word"][encoded_word]
            decoded_sentence += decoded_word
            decoded_sentence += " "
        
        decoded_sentence = decoded_sentence.strip()
        image_sentences.append(decoded_sentence)
        
    image.image_sentences = image_sentences
    
    if image.image_id in metadata["train_val_img_ids"]:
        image.is_train = True
        image.image_index = metadata["train_val_img_ids"].index(image.image_id)
        image.image_tensor_256 = train_images_256[image.image_index]
        image.image_tensor_128 = train_images_128[image.image_index]
        image.image_tensor_64 = train_images_64[image.image_index]      
    else:
        image.is_train = False
        image.image_index = metadata["test_img_ids"].index(image.image_id)
        image.image_tensor_256 = test_images_256[image.image_index]
        image.image_tensor_128 = test_images_128[image.image_index]
        image.image_tensor_64 = test_images_64[image.image_index]
        
    images_kaggle[image.image_id] = image

In [7]:
print(type(images_kaggle))
print(images_kaggle[18])

<class 'dict'>
Kaggle Image:
	Image Id: 18
	Image Class ID: 0
	Image Class Name: Black_footed_Albatross
	Image Sentences: ['light tan colored bird with a white head and an orange beak .', 'the bird has a very thick , curved , and beige beak', 'this bird has a long neck that is <UNKNOWN> and a pastel orange / blue narrow beak that <UNKNOWN> down at the tip', 'this bird is light brown , has a long hooked bill , and looks <UNKNOWN> .', 'this large white bird has a large curved bill and a brown eye', 'this bird is white with grey and has a long , pointy beak .', 'this bird is white with grey and has a long , pointy beak .', 'the crown of the bird is white , with light brown tones throughout .', 'the crown of the bird has distinctive tones of white and brown throughout .', 'this bird has a long neck and an orange bill']
	Is Train: False
	Image shapes:
		torch.Size([3, 64, 64])
		torch.Size([3, 128, 128])
		torch.Size([3, 256, 256])


### PAPER DATASET - IMPORT AND COMPILE IMAGES WITH THEIR IMAGE NUMBERS

In [8]:
images_res101 = sio.loadmat('/Users/lkk/Documents/BOUN CMPE/CMPE 537-Computer Vision/Term Project/Code/Datasets/CUB_Paper/res101.mat')

In [9]:
print(type(images_res101))
print(images_res101.keys())

<class 'dict'>
dict_keys(['__header__', '__version__', '__globals__', 'image_files', 'features', 'labels'])


In [10]:
for key in images_res101.keys():
    if not key.startswith("_"):
        print(f"Key: {key} Type: {type(images_res101[key])} Shape: {images_res101[key].shape}\n"
              f"First element type: {type(images_res101[key][0])}\n"
              f"Dype: {images_res101[key].dtype}\n"
              f"Element: {images_res101[key][0]}\n")

Key: image_files Type: <class 'numpy.ndarray'> Shape: (11788, 1)
First element type: <class 'numpy.ndarray'>
Dype: object
Element: [array(['/BS/Deep_Fragments/work/MSc/CUB_200_2011/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg'],
       dtype='<U126')                                                                                                                     ]

Key: features Type: <class 'numpy.ndarray'> Shape: (2048, 11788)
First element type: <class 'numpy.ndarray'>
Dype: float64
Element: [0.40892997 0.41969657 0.18802039 ... 0.30434826 0.35864624 0.65450728]

Key: labels Type: <class 'numpy.ndarray'> Shape: (11788, 1)
First element type: <class 'numpy.ndarray'>
Dype: uint8
Element: [151]



In [11]:
imageNumbers = []
for i in range(len(images_res101["image_files"])):
    filePath = images_res101["image_files"][i][0][0]
    imageNumber = int(filePath.split("_")[-1].split(".")[0])
    imageNumbers.append(imageNumber)

images_res101['imageNumbers'] = np.array(imageNumbers)

In [12]:
print(images_res101.keys())

dict_keys(['__header__', '__version__', '__globals__', 'image_files', 'features', 'labels', 'imageNumbers'])


In [13]:
attribute_splits = sio.loadmat("/Users/lkk/Documents/BOUN CMPE/CMPE 537-Computer Vision/Term Project/Code/Datasets/CUB_Paper/att_splits.mat")

In [14]:
print(attribute_splits.keys())

dict_keys(['__header__', '__version__', '__globals__', 'allclasses_names', 'att', 'original_att', 'test_seen_loc', 'test_unseen_loc', 'train_loc', 'trainval_loc', 'val_loc'])


In [15]:
for key in attribute_splits.keys():
    if not key.startswith("_"):
        print(f"Key: {key} Type: {type(attribute_splits[key])} Shape: {attribute_splits[key].shape}\n"
              f"First element type: {type(attribute_splits[key][0])}\n"
              f"Dype: {attribute_splits[key].dtype}\n"
              f"Element: {attribute_splits[key][0]}\n")

Key: allclasses_names Type: <class 'numpy.ndarray'> Shape: (200, 1)
First element type: <class 'numpy.ndarray'>
Dype: object
Element: [array(['002.Laysan_Albatross'], dtype='<U20')]

Key: att Type: <class 'numpy.ndarray'> Shape: (312, 200)
First element type: <class 'numpy.ndarray'>
Dype: float64
Element: [0.0106384  0.         0.         0.         0.         0.
 0.         0.         0.00474149 0.         0.0016366  0.
 0.         0.00490575 0.         0.         0.00187755 0.
 0.         0.         0.10176533 0.0046952  0.04470901 0.00189903
 0.01095034 0.         0.         0.0034344  0.         0.
 0.         0.         0.         0.00790824 0.00311325 0.
 0.         0.         0.00690833 0.00295739 0.01006911 0.00637794
 0.         0.         0.         0.         0.00904136 0.00937637
 0.00877098 0.04062457 0.01286689 0.         0.         0.
 0.         0.         0.         0.         0.0038029  0.
 0.         0.         0.01395605 0.00906423 0.01055845 0.
 0.00951459 0.      

In [16]:
label_indices = images_res101["labels"] - 1
labelNames = attribute_splits["allclasses_names"][label_indices[:, 0]]
np.info(labelNames)
print(labelNames[0])
labelNames[0]

class:  ndarray
shape:  (11788, 1)
strides:  (8, 8)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x158260000
byteorder:  little
byteswap:  False
type: object
[array(['001.Black_footed_Albatross'], dtype='<U26')]


array([array(['001.Black_footed_Albatross'], dtype='<U26')], dtype=object)

In [17]:
dataset_info = images_res101.copy()
dataset_info["labelNames"] = labelNames

In [18]:
print(dataset_info.keys())

dict_keys(['__header__', '__version__', '__globals__', 'image_files', 'features', 'labels', 'imageNumbers', 'labelNames'])


In [19]:
dataset_info

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Tue Apr 25 16:59:11 2017',
 '__version__': '1.0',
 '__globals__': [],
 'image_files': array([[array(['/BS/Deep_Fragments/work/MSc/CUB_200_2011/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg'],
               dtype='<U126')                                                                                                                     ],
        [array(['/BS/Deep_Fragments/work/MSc/CUB_200_2011/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0002_55.jpg'],
               dtype='<U122')                                                                                                                 ],
        [array(['/BS/Deep_Fragments/work/MSc/CUB_200_2011/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0003_796136.jpg'],
               dtype='<U126')                                                                                      

### DRAFT - COMBINE INFO FROM BOTH DATASETS

In [141]:
label_indices = images_res101["labels"] - 1
labelNames = attribute_splits["allclasses_names"][label_indices[:, 0]]
np.info(labelNames)
print(labelNames[0])
labelNames[0]

class:  ndarray
shape:  (11788, 1)
strides:  (8, 8)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x130568000
byteorder:  little
byteswap:  False
type: object
[array(['001.Black_footed_Albatross'], dtype='<U26')]


array([array(['001.Black_footed_Albatross'], dtype='<U26')], dtype=object)

In [178]:
print(images_kaggle[dataset_info["imageNumbers"][0]])
print(dataset_info["imageNumbers"][0])

Kaggle Image:
	Image Id: 796111
	Image Class ID: 0
	Image Class Name: Black_footed_Albatross
	Image Sentences: ['the medium sized bird has a dark grey color , a black downward curved beak , and long wings .', 'the bird is dark grey brown with a thick curved bill and a flat shaped tail .', 'bird has brown body feathers , white breast feathers and black beak', 'this bird has a dark brown overall body color , with a small white patch around the base of the bill .', 'the bird has very long and large brown wings , as well as a black body and a long black beak .', 'it is a type of albatross with black wings , tail , back and beak , and has a white ring at the base of its beak .', 'this bird has brown plumage and a white ring at the base of its long , curved brown beak .', 'the entire body is dark brown , as is the bill , with a white band encircling where the bill meets the head .', 'this bird is gray in color , with a large curved beak .', 'a large gray bird with a long wingspan and a long 

### PROJECT DATASET - COMBINE INFO FROM BOTH DATASETS

In [20]:
image_tensors_256 = []
image_tensors_128 = []
image_tensors_64 = []
sentence_descriptions = []
for imageNumber in dataset_info["imageNumbers"]:
    kaggleImage = images_kaggle[imageNumber]
    image_tensors_256.append(kaggleImage.image_tensor_256)
    image_tensors_128.append(kaggleImage.image_tensor_128)
    image_tensors_64.append(kaggleImage.image_tensor_64)
    sentence_descriptions.append(np.reshape(np.array(kaggleImage.image_sentences), newshape=(10,1)))
    
dataset_info["image_tensors_256"] = torch.stack(image_tensors_256)
dataset_info["image_tensors_128"] = torch.stack(image_tensors_128)
dataset_info["image_tensors_64"] = torch.stack(image_tensors_64)
dataset_info["sentence_descriptions"] = np.array(sentence_descriptions)

In [21]:
print(type(dataset_info["image_tensors_256"]))
dataset_info["image_tensors_256"].shape

<class 'torch.Tensor'>


torch.Size([11788, 3, 256, 256])

In [22]:
print(type(dataset_info["sentence_descriptions"]))
dataset_info["sentence_descriptions"].shape

<class 'numpy.ndarray'>


(11788, 10, 1)

In [23]:
dataset_info.keys()

dict_keys(['__header__', '__version__', '__globals__', 'image_files', 'features', 'labels', 'imageNumbers', 'labelNames', 'image_tensors_256', 'image_tensors_128', 'image_tensors_64', 'sentence_descriptions'])

In [24]:
print(sentence_descriptions[0])

[['the medium sized bird has a dark grey color , a black downward curved beak , and long wings .']
 ['the bird is dark grey brown with a thick curved bill and a flat shaped tail .']
 ['bird has brown body feathers , white breast feathers and black beak']
 ['this bird has a dark brown overall body color , with a small white patch around the base of the bill .']
 ['the bird has very long and large brown wings , as well as a black body and a long black beak .']
 ['it is a type of albatross with black wings , tail , back and beak , and has a white ring at the base of its beak .']
 ['this bird has brown plumage and a white ring at the base of its long , curved brown beak .']
 ['the entire body is dark brown , as is the bill , with a white band encircling where the bill meets the head .']
 ['this bird is gray in color , with a large curved beak .']
 ['a large gray bird with a long wingspan and a long black beak .']]


In [26]:
with open("/Users/lkk/Documents/BOUN CMPE/CMPE 537-Computer Vision/Term Project/Code/Datasets/CUB_Paper/produced/dataset_info.pkl", 'wb') as f:
    pickle.dump(dataset_info, f)

In [201]:
sio.savemat("/Users/lkk/Documents/BOUN CMPE/CMPE 537-Computer Vision/Term Project/Code/Datasets/CUB_Paper/produced/dataset_draft.mat", dataset_info)

In [200]:
np.info(dataset_info["features"])

class:  ndarray
shape:  (2048, 11788)
strides:  (8, 16384)
itemsize:  8
aligned:  True
contiguous:  False
fortran:  True
data pointer: 0x15d38c020
byteorder:  little
byteswap:  False
type: float64


In [187]:
count = 0
for image in images_kaggle.values():
    if len(image.image_sentences) != 10:
        print(len(image.image_sentences))
    else:
        count += 1
        
print(count)

11788


In [86]:
attribute_splits = sio.loadmat("/Users/lkk/Documents/BOUN CMPE/CMPE 537-Computer Vision/Term Project/Code/Datasets/CUB_Paper/att_splits.mat")

In [88]:
print(attribute_splits.keys())

dict_keys(['__header__', '__version__', '__globals__', 'allclasses_names', 'att', 'original_att', 'test_seen_loc', 'test_unseen_loc', 'train_loc', 'trainval_loc', 'val_loc'])


In [92]:
for key in attribute_splits.keys():
    if not key.startswith("_"):
        print(f"Key: {key} Type: {type(attribute_splits[key])} Shape: {attribute_splits[key].shape}\n"
              f"First element type: {type(attribute_splits[key][0])}\n"
              f"Dype: {attribute_splits[key].dtype}\n"
              f"Element: {attribute_splits[key][0]}\n")

Key: allclasses_names Type: <class 'numpy.ndarray'> Shape: (200, 1)
First element type: <class 'numpy.ndarray'>
Dype: object
Element: [array(['002.Laysan_Albatross'], dtype='<U20')]

Key: att Type: <class 'numpy.ndarray'> Shape: (312, 200)
First element type: <class 'numpy.ndarray'>
Dype: float64
Element: [0.0106384  0.         0.         0.         0.         0.
 0.         0.         0.00474149 0.         0.0016366  0.
 0.         0.00490575 0.         0.         0.00187755 0.
 0.         0.         0.10176533 0.0046952  0.04470901 0.00189903
 0.01095034 0.         0.         0.0034344  0.         0.
 0.         0.         0.         0.00790824 0.00311325 0.
 0.         0.         0.00690833 0.00295739 0.01006911 0.00637794
 0.         0.         0.         0.         0.00904136 0.00937637
 0.00877098 0.04062457 0.01286689 0.         0.         0.
 0.         0.         0.         0.         0.0038029  0.
 0.         0.         0.01395605 0.00906423 0.01055845 0.
 0.00951459 0.      

In [93]:
allclasses_names = attribute_splits["allclasses_names"]
print(allclasses_names)


[[array(['002.Laysan_Albatross'], dtype='<U20')]
 [array(['003.Sooty_Albatross'], dtype='<U19')]
 [array(['005.Crested_Auklet'], dtype='<U18')]
 [array(['007.Parakeet_Auklet'], dtype='<U19')]
 [array(['010.Red_winged_Blackbird'], dtype='<U24')]
 [array(['011.Rusty_Blackbird'], dtype='<U19')]
 [array(['012.Yellow_headed_Blackbird'], dtype='<U27')]
 [array(['013.Bobolink'], dtype='<U12')]
 [array(['015.Lazuli_Bunting'], dtype='<U18')]
 [array(['016.Painted_Bunting'], dtype='<U19')]
 [array(['017.Cardinal'], dtype='<U12')]
 [array(['018.Spotted_Catbird'], dtype='<U19')]
 [array(['019.Gray_Catbird'], dtype='<U16')]
 [array(['020.Yellow_breasted_Chat'], dtype='<U24')]
 [array(['021.Eastern_Towhee'], dtype='<U18')]
 [array(['022.Chuck_will_Widow'], dtype='<U20')]
 [array(['024.Red_faced_Cormorant'], dtype='<U23')]
 [array(['025.Pelagic_Cormorant'], dtype='<U21')]
 [array(['026.Bronzed_Cowbird'], dtype='<U19')]
 [array(['027.Shiny_Cowbird'], dtype='<U17')]
 [array(['028.Brown_Creeper'], dtype

In [140]:
print(allclasses_names[0])

[array(['002.Laysan_Albatross'], dtype='<U20')]


In [131]:
print(allclasses_names[0][0][0])

002.Laysan_Albatross


In [183]:
sss = np.reshape(np.array(images_kaggle[18].image_sentences), newshape=(10,1))
# sss = np.array(images_kaggle[18].image_sentences)
np.info(sss)
sss

class:  ndarray
shape:  (10, 1)
strides:  (452, 452)
itemsize:  452
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x138016600
byteorder:  little
byteswap:  False
type: <U113


array([['light tan colored bird with a white head and an orange beak .'],
       ['the bird has a very thick , curved , and beige beak'],
       ['this bird has a long neck that is <UNKNOWN> and a pastel orange / blue narrow beak that <UNKNOWN> down at the tip'],
       ['this bird is light brown , has a long hooked bill , and looks <UNKNOWN> .'],
       ['this large white bird has a large curved bill and a brown eye'],
       ['this bird is white with grey and has a long , pointy beak .'],
       ['this bird is white with grey and has a long , pointy beak .'],
       ['the crown of the bird is white , with light brown tones throughout .'],
       ['the crown of the bird has distinctive tones of white and brown throughout .'],
       ['this bird has a long neck and an orange bill']], dtype='<U113')

In [108]:
np.info(allclasses_names)
print(type(allclasses_names[0]))
np.info(allclasses_names[0])

class:  ndarray
shape:  (200, 1)
strides:  (8, 1600)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x12d0d2400
byteorder:  little
byteswap:  False
type: object
<class 'numpy.ndarray'>
class:  ndarray
shape:  (1,)
strides:  (1600,)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x12d0d2400
byteorder:  little
byteswap:  False
type: object


In [121]:
np.info(allclasses_names[label_indices[:,0]])
allclasses_names[label_indices[:,0]]

class:  ndarray
shape:  (11788, 1)
strides:  (8, 8)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x130ac0000
byteorder:  little
byteswap:  False
type: object


array([[array(['001.Black_footed_Albatross'], dtype='<U26')],
       [array(['001.Black_footed_Albatross'], dtype='<U26')],
       [array(['001.Black_footed_Albatross'], dtype='<U26')],
       ...,
       [array(['200.Common_Yellowthroat'], dtype='<U23')],
       [array(['200.Common_Yellowthroat'], dtype='<U23')],
       [array(['200.Common_Yellowthroat'], dtype='<U23')]], dtype=object)

In [97]:
print(type(images_res101["labels"]))

<class 'numpy.ndarray'>


In [101]:
label_indices = images_res101["labels"] - 1
np.info(label_indices)
print(np.max(label_indices))
print(np.min(label_indices))

class:  ndarray
shape:  (11788, 1)
strides:  (1, 1)
itemsize:  1
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x123220e00
byteorder:  little
byteswap:  False
type: uint8
199
0


In [99]:
label_indices_2 = images_res101["labels"] - 1
np.info(label_indices_2)

class:  ndarray
shape:  (11788, 1)
strides:  (1, 1)
itemsize:  1
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x138849400
byteorder:  little
byteswap:  False
type: uint8


In [118]:
np.info(label_indices[:, 0])
label_indices[:, 0]

class:  ndarray
shape:  (11788,)
strides:  (1,)
itemsize:  1
aligned:  True
contiguous:  True
fortran:  True
data pointer: 0x123220e00
byteorder:  little
byteswap:  False
type: uint8


array([150, 150, 150, ..., 149, 149, 149], dtype=uint8)

In [120]:
print(label_indices)

[[150]
 [150]
 [150]
 ...
 [149]
 [149]
 [149]]


In [100]:
print(np.array_equal(label_indices, label_indices_2)) 

True


In [105]:
print(label_indices)

[[150]
 [150]
 [150]
 ...
 [149]
 [149]
 [149]]


In [85]:
print(type(images_res101["labels"]))

<class 'numpy.ndarray'>


In [65]:
print(images_res101["imageNumbers"])

[796111, 55, 796136, 796090, 796065, 796138, 796083, 34, 796097, 89, 796067, 796098, 796104, 796059, 796089, 796057, 796095, 100, 796115, 796086, 796140, 796127, 796120, 212, 796132, 796066, 796108, 796071, 796129, 18, 796064, 796063, 796125, 796103, 796109, 796078, 796106, 796074, 796076, 796082, 796141, 796101, 796068, 170, 796135, 796139, 796113, 59, 417, 796114, 796126, 796122, 796096, 426, 796121, 92, 796062, 796133, 796069, 796077, 545, 1027, 1033, 930, 565, 702, 696, 910, 614, 492, 737, 1059, 571, 643, 482, 658, 628, 876, 699, 924, 472, 801, 784, 619, 918, 870, 1020, 543, 570, 500, 637, 488, 777, 563, 674, 809, 726, 788, 792, 927, 668, 671, 506, 821, 800, 524, 756, 564, 883, 602, 834, 725, 1013, 673, 621, 869, 735, 611, 504, 630, 1071, 796395, 1078, 796366, 796342, 796390, 796372, 796355, 796402, 796373, 1075, 796349, 796391, 796359, 796339, 796398, 796401, 1161, 796361, 796357, 1122, 1066, 1149, 1128, 1154, 796387, 1065, 796375, 796364, 1210, 1076, 1105, 1162, 1211, 1130, 79635

In [10]:
print(len(res101["labels"]))

11788


In [24]:
labelsTensor = torch.tensor(res101["labels"])

In [25]:
labelsTensor.shape

torch.Size([11788, 1])

In [26]:
print(torch.max(labelsTensor))
print(torch.min(labelsTensor))


tensor(200, dtype=torch.uint8)
tensor(1, dtype=torch.uint8)


In [5]:
print(res101.keys())

dict_keys(['__header__', '__version__', '__globals__', 'image_files', 'features', 'labels'])


In [15]:
print(res101["features"])

[[0.40892997 0.41969657 0.18802039 ... 0.30434826 0.35864624 0.65450728]
 [0.25413808 0.30752203 0.08614247 ... 0.19108224 0.13533165 0.20017533]
 [2.12168241 0.77530569 2.11704683 ... 0.11788042 0.06038139 0.00543019]
 ...
 [0.11358754 0.21275087 0.04323977 ... 0.35180986 0.54067051 0.23211631]
 [0.04199803 0.14942844 0.19070306 ... 0.         0.35444796 0.03051368]
 [0.12280495 0.34912837 0.         ... 0.04952463 0.03132629 0.        ]]


In [16]:
print(len(res101["features"]))

2048


In [27]:
featuresTensor = torch.tensor(res101["features"])

In [28]:
featuresTensor.shape

torch.Size([2048, 11788])

In [29]:
print(torch.max(featuresTensor))
print(torch.min(featuresTensor))


tensor(35.9458, dtype=torch.float64)
tensor(0., dtype=torch.float64)


In [30]:
print(featuresTensor[:, 0])

tensor([0.4089, 0.2541, 2.1217,  ..., 0.1136, 0.0420, 0.1228],
       dtype=torch.float64)


In [31]:
featuresTensor[:, 1].shape

torch.Size([2048])

In [5]:
print(res101.keys())

dict_keys(['__header__', '__version__', '__globals__', 'image_files', 'features', 'labels'])


In [22]:
print(res101["image_files"])

[[array(['/BS/Deep_Fragments/work/MSc/CUB_200_2011/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg'],
        dtype='<U126')                                                                                                                     ]
 [array(['/BS/Deep_Fragments/work/MSc/CUB_200_2011/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0002_55.jpg'],
        dtype='<U122')                                                                                                                 ]
 [array(['/BS/Deep_Fragments/work/MSc/CUB_200_2011/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0003_796136.jpg'],
        dtype='<U126')                                                                                                                     ]
 ...
 [array(['/BS/Deep_Fragments/work/MSc/CUB_200_2011/CUB_200_2011/images/200.Common_Yellowthroat/Common_Yellowthroat_0122_190570.jpg'],
        dtype='<U120')    

In [23]:
print(len(res101["image_files"]))

11788


In [50]:
print(res101["image_files"][0][0][0])

/BS/Deep_Fragments/work/MSc/CUB_200_2011/CUB_200_2011/images/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg


In [56]:
filePath = res101["image_files"][2][0][0]
imageNumber = int(filePath.split("_")[-1].split(".")[0])
print(imageNumber)

796136


In [60]:
imageNumbers = []
for i in range(len(res101["image_files"])):
    filePath = res101["image_files"][i][0][0]
    imageNumber = int(filePath.split("_")[-1].split(".")[0])
    imageNumbers.append(imageNumber)
    
print(len(imageNumbers))
print(imageNumbers[0:3], imageNumbers[-3:])

11788
[796111, 55, 796136] [190570, 190902, 190407]


In [46]:
print(type(res101["image_files"][0][0][0]))

<class 'numpy.str_'>


In [17]:
labelsTensor = torch.tensor(res101["features"])

In [18]:
labelsTensor.shape

torch.Size([2048, 11788])

In [19]:
print(torch.max(labelsTensor))
print(torch.min(labelsTensor))


tensor(35.9458, dtype=torch.float64)
tensor(0., dtype=torch.float64)


In [20]:
print(labelsTensor[:, 0])

tensor([0.4089, 0.2541, 2.1217,  ..., 0.1136, 0.0420, 0.1228],
       dtype=torch.float64)


In [21]:
labelsTensor[:, 1].shape

torch.Size([2048])